## Setup

In [ ]:
# !pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# TODO: Add image explaining the dataset

In [ ]:
# TODO: Edit config to have correct model with https://schema.org/docs/jsonldcontext.json context (this needs to be created)

In [ ]:
# TODO: working in sandbox https://sandbox.bluebrainnexus.io/web

In [ ]:
PROJECT = "YOUR_PROJECT" # TODO: Add your project

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
forge = KnowledgeGraphForge("./data/configuration/config.yml", token=TOKEN, bucket=f"tutorialnexus/{PROJECT}")

## Data

In [ ]:
!cd data/raw && curl -s -O http://files.grouplens.org/datasets/movielens/ml-latest-small.zip && unzip -qq ml-latest-small.zip && cd ml-latest-small && ls

In [ ]:
directory = "./data/raw/ml-latest-small"

In [ ]:
movies_df = pd.read_csv(f"{directory}/movies.csv")

In [ ]:
ratings_df = pd.read_csv(f"{directory}/ratings.csv", dtype={"movieId":"string"})

In [ ]:
tags_df = pd.read_csv(f"{directory}/tags.csv", dtype={"movieId":"string"})

In [ ]:
links_df = pd.read_csv(f"{directory}/links.csv")

In [ ]:
movies_links_df = pd.merge(movies_df, links_df, on='movieId')

## Resources

In [ ]:
movies_links_df["type"] = "Movie"

In [ ]:
ratings_df["type"] = "Rating"

In [ ]:
tags_df["type"] = "Tag"

In [ ]:
movies_links_df["id"] = movies_links_df["movieId"].apply(lambda x: forge.format("identifier", "movies", x))

In [ ]:
movies_links_df["genres"] = movies_links_df["genres"].apply(lambda x: x.split("|"))

In [ ]:
ratings_df["movieId.id"] = movies_links_df["movieId"].apply(lambda x: forge.format("identifier", "movies", x))

In [ ]:
tags_df["movieId.id"] = tags_df["movieId"].apply(lambda x: forge.format("identifier", "movies", x))

In [ ]:
movies_resources = forge.from_dataframe(movies_links_df, np.nan, ".")

In [ ]:
ratings_resources = forge.from_dataframe(ratings_df, np.nan, ".")

In [ ]:
tags_resources = forge.from_dataframe(tags_df, np.nan, ".")

In [ ]:
print(movies_resources[0])

In [ ]:
print(ratings_resources[0])

In [ ]:
print(tags_resources[629])

## Register

In [ ]:
forge.register(movies_resources[25:30])

In [ ]:
forge.register(ratings_resources)

In [ ]:
forge.register(tags_resources)

## Query

In [ ]:
query = """
    SELECT ?id
    WHERE {
        ?id a Movie ;
              <http://test.org/genres> "Fantasy" .
    }
"""

In [ ]:
resources = forge.sparql(query, limit=10, debug=True)

In [ ]:
movie = forge.retrieve(resources[0].id)

In [ ]:
print(movie)

In [ ]:
movie._store_metadata